In [ ]:
import os
import pandas as pd
import geopandas as gpd

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)

CRS_NYC = "EPSG:2263"

DATA_RAW = os.path.join("..", "data", "raw")
DATA_PROCESSED = os.path.join("..", "data", "processed")
os.makedirs(DATA_PROCESSED, exist_ok=True)

PATH_BUILDINGS = os.path.join(DATA_RAW, "building_current", "nyc_building_footprints_current.zip")
PATH_NTA = os.path.join(DATA_RAW, "nta", "nyc_nta_2020.zip")

In [2]:
buildings = gpd.read_file(f"zip://{PATH_BUILDINGS}")
buildings = buildings.to_crs(CRS_NYC)

buildings["footprint_sqft"] = buildings.geometry.area

buildings[["footprint_sqft"]].describe()


,footprint_sqft
count,1.082999e+06
mean,1.609370e+03
std,5.663732e+03
min,3.247651e+01
25%,6.539524e+02
50%,9.593253e+02
75%,1.323959e+03
max,1.171875e+06


In [3]:
nta = gpd.read_file(f"zip://{PATH_NTA}")

nta = nta.rename(columns={
    "nta2020": "NTACode",
    "ntaname": "NTAName"
})

nta = nta[["NTACode", "NTAName", "boroname", "geometry"]]
nta = nta.to_crs(CRS_NYC)

nta["nta_area_sqft"] = nta.geometry.area
nta["nta_area_sqmi"] = nta["nta_area_sqft"] / 27_878_400

nta.head()

,NTACode,NTAName,boroname,geometry,nta_area_sqft,nta_area_sqmi
0,BK0101,Greenpoint,Brooklyn,"POLYGON ((1003059.997 204572.025, 1002991.367 ...",3.532174e+07,1.266993
1,BK0102,Williamsburg,Brooklyn,"POLYGON ((995851.916 203199.332, 995969.193 20...",2.885280e+07,1.034952
2,BK0103,South Williamsburg,Brooklyn,"POLYGON ((998047.21 196303.325, 998157.901 196...",1.520896e+07,0.545546
3,BK0104,East Williamsburg,Brooklyn,"POLYGON ((1005302.497 199455.73, 1005307.792 1...",5.226747e+07,1.874838
4,BK0201,Brooklyn Heights,Brooklyn,"POLYGON ((986367.736 190549.239, 985813.836 19...",9.982088e+06,0.358058


In [4]:
sample = buildings.sample(10_000, random_state=42)

sample_join = gpd.sjoin(
    sample,
    nta,
    how="left",
    predicate="intersects"
)

sample_join["NTACode"].notna().mean()

np.float64(0.9999000499750125)

In [5]:
buildings_nta = gpd.sjoin(
    buildings,
    nta,
    how="left",
    predicate="intersects"
).drop(columns=["index_right"])

In [6]:
nta_density = (
    buildings_nta
    .groupby(["NTACode", "NTAName", "boroname"])
    .agg(
        building_count=("footprint_sqft", "count"),
        total_footprint_sqft=("footprint_sqft", "sum"),
        median_building_sqft=("footprint_sqft", "median")
    )
    .reset_index()
)

nta_density.head()

,NTACode,NTAName,boroname,building_count,total_footprint_sqft,median_building_sqft
0,BK0101,Greenpoint,Brooklyn,5147,1.340080e+07,1248.021588
1,BK0102,Williamsburg,Brooklyn,3646,1.037733e+07,1334.824761
2,BK0103,South Williamsburg,Brooklyn,2294,5.418461e+06,1408.694795
3,BK0104,East Williamsburg,Brooklyn,4939,1.745002e+07,1354.539352
4,BK0201,Brooklyn Heights,Brooklyn,1492,3.913004e+06,1358.771123


In [7]:
nta_density = nta.merge(
    nta_density,
    on=["NTACode", "NTAName", "boroname"],
    how="left"
)

nta_density[[
    "building_count",
    "total_footprint_sqft",
    "median_building_sqft"
]] = nta_density[[
    "building_count",
    "total_footprint_sqft",
    "median_building_sqft"
]].fillna(0)

nta_density["built_area_ratio"] = (
    nta_density["total_footprint_sqft"] / nta_density["nta_area_sqft"]
)

nta_density["buildings_per_sqmi"] = (
    nta_density["building_count"] / nta_density["nta_area_sqmi"]
)

nta_density["built_sqft_per_sqmi"] = (
    nta_density["total_footprint_sqft"] / nta_density["nta_area_sqmi"]
)

In [8]:
nta_density["built_area_ratio"].describe()

count    262.000000
mean       0.224841
std        0.134573
min        0.000000
25%        0.145247
50%        0.245855
75%        0.323794
max        0.549023
Name: built_area_ratio, dtype: float64

In [9]:
nta_density.sort_values("built_area_ratio", ascending=False).head(10)

,NTACode,NTAName,boroname,geometry,nta_area_sqft,nta_area_sqmi,building_count,total_footprint_sqft,median_building_sqft,built_area_ratio,buildings_per_sqmi,built_sqft_per_sqmi
130,MN0502,Midtown-Times Square,Manhattan,"POLYGON ((991725.244 217725.299, 992169.505 21...",2.455254e+07,0.880701,1444.0,1.347990e+07,5031.327741,0.549023,1639.602531,1.530587e+07
129,MN0501,Midtown South-Flatiron-Union Square,Manhattan,"POLYGON ((989555.974 211702.854, 989420.216 21...",1.487902e+07,0.533711,1306.0,7.998146e+06,3405.174031,0.537545,2447.015022,1.498590e+07
121,MN0201,SoHo-Little Italy-Hudson Square,Manhattan,"POLYGON ((983469.159 204638.902, 983496.09 204...",1.291676e+07,0.463325,1662.0,6.500400e+06,2274.769680,0.503253,3587.114384,1.402989e+07
134,MN0604,East Midtown-Turtle Bay,Manhattan,"POLYGON ((995598.36 215579.024, 995626.716 215...",1.313809e+07,0.471264,1101.0,6.393684e+06,2142.036303,0.486652,2336.268575,1.356709e+07
128,MN0402,Hell's Kitchen,Manhattan,"POLYGON ((985324.423 221001.9, 985817.17 22072...",1.838232e+07,0.659375,1666.0,8.732165e+06,1972.715628,0.475031,2526.635671,1.324310e+07
140,MN0802,Upper East Side-Carnegie Hill,Manhattan,"POLYGON ((998281.392 225347.968, 998145.231 22...",2.006536e+07,0.719746,2679.0,9.498898e+06,1908.545717,0.473398,3722.146907,1.319757e+07
122,MN0202,Greenwich Village,Manhattan,"POLYGON ((987048.27 206851.147, 987057.852 206...",1.060046e+07,0.380239,1224.0,4.996409e+06,2088.616231,0.471339,3219.027560,1.314018e+07
132,MN0602,Gramercy,Manhattan,"POLYGON ((990196.892 207745.371, 990187.613 20...",7.526000e+06,0.269958,818.0,3.486676e+06,1996.561202,0.463284,3030.099995,1.291562e+07
127,MN0401,Chelsea-Hudson Yards,Manhattan,"POLYGON ((983754.619 217001.185, 983857.657 21...",2.967175e+07,1.064328,2075.0,1.318783e+07,2007.850335,0.444458,1949.587507,1.239076e+07
119,MN0102,Tribeca-Civic Center,Manhattan,"POLYGON ((984440.604 200699.422, 984402.913 20...",1.357825e+07,0.487053,902.0,5.706854e+06,2960.023311,0.420294,1851.955068,1.171711e+07


In [10]:
borough_summary = (
    nta_density
    .groupby("boroname", as_index=False)
    .agg(
        avg_built_ratio=("built_area_ratio", "mean"),
        total_building_sqft=("total_footprint_sqft", "sum"),
        total_buildings=("building_count", "sum"),
        nta_count=("NTACode", "nunique")
    )
)

borough_summary

,boroname,avg_built_ratio,total_building_sqft,total_buildings,nta_count
0,Bronx,0.212835,2.319491e+08,104508.0,50
1,Brooklyn,0.253432,5.225670e+08,330203.0,69
2,Manhattan,0.334825,2.102427e+08,44981.0,38
3,Queens,0.189133,5.925204e+08,461139.0,82
4,Staten Island,0.110763,1.893624e+08,142318.0,23


In [11]:
out_path = os.path.join(DATA_PROCESSED, "nta_construction_density.parquet")
nta_density.to_parquet(out_path, index=False)

print("Saved:", out_path)

Saved: ..\data\processed\nta_construction_density.parquet
